In [ ]:
var link_peticoes = 'http://127.0.0.1:8000/api/forma_modelo?cnj=' + $('#txtAutos_ConsultaAutos').val()

In [97]:
import requests
import datetime

a = datetime.datetime.now()
s = requests.session()

u = 'http://127.0.0.1:8000/api/forma_modelo?cnj=0002990-19.2011.8.26.0053'

# params = {'cnj':'0002990-19.2011.8.26.0053'}

r = s.get(u, params=params)
r.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8">\n  <meta name="robots" content="NONE,NOARCHIVE">\n  <title>KeyError\n          at /api/forma_modelo</title>\n  <style type="text/css">\n    html * { padding:0; margin:0; }\n    body * { padding:10px 20px; }\n    body * * { padding:0; }\n    body { font:small sans-serif; background-color:#fff; color:#000; }\n    body>div { border-bottom:1px solid #ddd; }\n    h1 { font-weight:normal; }\n    h2 { margin-bottom:.8em; }\n    h3 { margin:1em 0 .5em 0; }\n    h4 { margin:0 0 .5em 0; font-weight: normal; }\n    code, pre { font-size: 100%; white-space: pre-wrap; }\n    table { border:1px solid #ccc; border-collapse: collapse; width:100%; background:white; }\n    tbody td, tbody th { vertical-align:top; padding:2px 3px; }\n    thead th {\n      padding:1px 6px 1px 3px; background:#fefefe; text-align:left;\n      font-weight:normal; font-size:11px; border:1px solid #ddd;\n    }\n   

In [100]:
from bs4 import BeautifulSoup as bs
import datetime
import re
import json
from unidecode import unidecode
import io
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from api.models import *
from django.conf import settings
import os
import textract
import tempfile
import time
import requests
from seleniumrequests import Chrome

def get_siajd_looged_session():
    s = requests.session()
    url_login = 'https://siajd.pgm.prefeitura.sp.gov.br/UC_AUT_001_AutenticarUsuario/Login'

    data = {'Usuario': 'D858414',
            'Senha': '013452Ab',
            'TextoCaptcha': ''}

    s.post(url_login, data=data)

    return s


def get_siajd(s, cnj):

    url = 'https://siajd.pgm.prefeitura.sp.gov.br/UC_ACA_024_ConsultarAutos/Pesquisar'
    params = {'NumeroAutos': cnj}

    r = s.get(url, params=params, allow_redirects=True)
    sp = bs(r.content, 'html5lib')

    return {unidecode(i.get('title').lower().strip()).replace(' ', '_'): i.get('value') for i in sp.findAll(attrs={'class':'form-control'})}

s = get_siajd_looged_session()

cnj = '0003081-80.2009.8.26.0053'

url = 'https://siajd.pgm.prefeitura.sp.gov.br/UC_ACA_024_ConsultarAutos/Pesquisar'
params = {'NumeroAutos': cnj}

r = s.get(url, params=params, allow_redirects=True)
sp = bs(r.content, 'html5lib')


In [102]:
{unidecode(i.get('title').lower().strip()).replace(' ', '_'): i.get('value') for i in sp.findAll(attrs={'class':'form-control'})}

{'ndeg_dos_autos_unificado': '0003081-80.2009.8.26.0053',
 'ndeg_dos_autos': '0003081-80.2009.8.26.0053',
 'procurador_responsavel': 'ANA LÚCIA MARINO ROSSO',
 'unidade': 'JUD 5',
 'vara': '11ª Vara de Fazenda Pública',
 'objeto': 'Migração do Legado',
 'classe_cnj': '',
 'complemento_do_objeto': 'ANULAÇÃO DE DÉBITO NÃO TRIBUTÁRIO/MULTA DE TRÂNSITO',
 'assunto_cnj': '',
 'numero_pa': '2009-0.053.152-1',
 'numero_sei': '',
 'autor_principal': 'BANCO ITAU S.A.',
 'possui_oc_nao_revisada': 'Não',
 'reu_principal': 'Município de São Paulo',
 'observacao': 'DEFERIDA A ANTECIPAÇÃO DE TUTELA'}

In [95]:
import io
from docx import Document

modelo = Modelos.objects.get(pk=3)


# chaves = {'vara': '7ª Vara de Fazenda Pública', 'foro': 'dasdsa', 'comarca': 'sdadsa', 'autor_principal': 'RENATO JOSÉ MARABOLIM TRANSPORTES - ME', 'cnj': '0002990-19.2011.8.26.0053'}

chaves = {'vara': '7ª Vara de Fazenda Pública', 'cnj': '0002990-19.2011.8.26.0053', 'autor_principal': 'RENATO JOSÉ MARABOLIM TRANSPORTES - ME'}


buffer = io.BytesIO(modelo.bytes)
doc = Document(buffer)

coleta = []
coletar = False

x = []

for paragrafo in doc.paragraphs:
    for run in paragrafo.runs:
        if '{' in run.text:
            coletar = True
            coleta.append(run)
            
        elif coletar:
            coleta.append(run)
            
        if '}' in run.text:

            x.append(coleta)
            coletar = False                
            key = ''.join([i.text for i in coleta])
            
            key = re.search('{(.*)}', key).group(1)

            if '}' in coleta[0].text:
                coleta[0].text = re.sub('{.*?}',chaves[key].upper(), coleta[0].text) 
            else:
                coleta[0].text = re.sub('{.*',chaves[key].upper(), coleta[0].text) 
                
            for remover in coleta[1:]:
                remover.clear()

            coleta = []



                
                
                
                

            


        
        
                
            
            



KeyError: ' batatinha '

In [94]:
a = coleta[0].text
re.sub('{.*?}','aaa', a)

'EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(ÍZA) DE DIREITO DA aaa – FORO CENTRAL – COMARCA DA CAPITAL'

In [81]:
key


'EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(ÍZA) DE DIREITO DA vara – FORO CENTRAL – COMARCA DA CAPITAL'

In [4]:
import pandas as pd
import re
df = pd.read_excel('/home/ezequiel/Downloads/acoes_recentes.xlsx')

a = list(df['vara'].unique())
[i for i in a if 'Precatórias' in i]





['Setor Unificado de Cartas Precatórias Cíveis - Setor de Cartas Precatórias Cíveis - Cap']

In [10]:
from api.models import *
unidade = Unidade.objects.get(nome='JUD 5')
query_modelos = Modelos.objects.filter(unidade=unidade).values('id', 'nome')
query_modelos

<QuerySet [{'id': 1, 'nome': 'Carro Colado'}]>

In [17]:
import requests

s = requests.session()
url_login = 'https://siajd.pgm.prefeitura.sp.gov.br/UC_AUT_001_AutenticarUsuario/Login'

data = {'Usuario': 'D858414',
        'Senha': '013451Ab',
        'TextoCaptcha': ''}

s.post(url_login, data=data)

u = 'https://siajd.pgm.prefeitura.sp.gov.br/UC_ACA_017_ManterAcaoJudicial/ImportarDadosTJ'

data = {'NrAutos': '1063139-80.2019.8.26.0053'}

r = s.post(u, json=data)

r.json()

{'Tribunal': 1,
 'Foro': 38,
 'Vara': 728,
 'Competencia': 28,
 'Classe': 157,
 'Assunto': 474,
 'ListaPartes': [{'CdParteAcao': 0,
   'CPFCNPJ': '187.416.958-60',
   'CdTipoParte': 0,
   'TipoParte': '',
   'CdRelevancia': 3,
   'Relevancia': 'Não Definida',
   'Parte': 'Andrea Silva Ferreira',
   'CdPessoa': 417131,
   'CdVersaoPessoa': 0},
  {'CdParteAcao': 0,
   'CPFCNPJ': '',
   'CdTipoParte': 0,
   'TipoParte': '',
   'CdRelevancia': 3,
   'Relevancia': 'Não Definida',
   'Parte': 'Prefeitura do Municipio de São Paulo',
   'CdPessoa': 417442,
   'CdVersaoPessoa': 0}],
 'ValorCausa': '4.994,72',
 'OutrosNumeros': [],
 'SegredoJustica': 'N',
 'MensagemInformativa': ''}

In [5]:
import requests
import datetime

a = datetime.datetime.now()
s = requests.session()

u = 'https://pgmsp.herokuapp.com/api/consulta_processo'

data = {'nuprocessounificadoformatado':'0002990-19.2011.8.26.0053'}

r = s.post(u, data=data)
b = datetime.datetime.now()
b - a

datetime.timedelta(0, 28, 290173)

In [26]:
import docx

file = '/home/ezequiel/Desktop/DIS_Processo (SEI).docx'
f = open(file, 'rb')
x = f.read()


y = open('/home/ezequiel/Desktop/t.txt', 'wb')
y.write(x)
y.close()


In [30]:
z = open('/home/ezequiel/Desktop/t.txt', 'rb')


d = docx.Document(z)

for paragraphs in d.paragraphs:
    print(paragraphs.text)


Documento de Interface
Webservice Processo (SEI)<Nome do Projeto>

					
	
Revisões do Documento
		
				

Sumário








Objetivo 
Descrever tecnicamente o funcionamento do Webservice do Sistema Municipal de Processos - SEI, que disponibiliza várias operações para a consulta, o cadastro, a alteração e a remoção de dados dos processos no SEI. 
O Webservice Processo (SEI) estará disponível aos demais sistemas através de chamadas SOAP (Simple Object Access Protocol), com troca de mensagens de requisição e de resposta em formato XML.












 


Forma de Integração




Interfaces de Comunicação
Parâmetros de Entrada e Saída
AutuarProcessoEletronico




ConsultarProcesso


InserirDocumento.v1.0


ConsultarTiposProcesso


ConsultarUnidades


ConcluirProcesso


TramitarProcesso


ReabrirProcesso




ConsultarDocumentosDoProcesso



BloquearAssinaturaDocumento



ConsultarPermissoesDoUsuario


ConsultarUsuarios


CconsultarUnidadesDoOrgao


ConsultarOrgaos


RelacionarProcesso


Consultar

In [35]:
import io
u = io.BytesIO(x)
q = docx.Document(u)

for p in q.paragraphs:
    print(p.text)

Documento de Interface
Webservice Processo (SEI)<Nome do Projeto>

					
	
Revisões do Documento
		
				

Sumário








Objetivo 
Descrever tecnicamente o funcionamento do Webservice do Sistema Municipal de Processos - SEI, que disponibiliza várias operações para a consulta, o cadastro, a alteração e a remoção de dados dos processos no SEI. 
O Webservice Processo (SEI) estará disponível aos demais sistemas através de chamadas SOAP (Simple Object Access Protocol), com troca de mensagens de requisição e de resposta em formato XML.












 


Forma de Integração




Interfaces de Comunicação
Parâmetros de Entrada e Saída
AutuarProcessoEletronico




ConsultarProcesso


InserirDocumento.v1.0


ConsultarTiposProcesso


ConsultarUnidades


ConcluirProcesso


TramitarProcesso


ReabrirProcesso




ConsultarDocumentosDoProcesso



BloquearAssinaturaDocumento



ConsultarPermissoesDoUsuario


ConsultarUsuarios


CconsultarUnidadesDoOrgao


ConsultarOrgaos


RelacionarProcesso


Consultar

<_io.BufferedReader name='/home/ezequiel/Desktop/t.txt'>

In [6]:
from bs4 import BeautifulSoup as bs
import datetime
import re
import json
from unidecode import unidecode
import io
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from api.models import *
from django.conf import settings
import os
import textract
import tempfile
import time
import requests
from seleniumrequests import Chrome


nuprocessounificadoformatado = '1058720-17.2019.8.26.0053'
documentos=False
driver=False


crit_assinatura = re.compile('Este documento é cópia do original.*? e código .{,8}\.',re.DOTALL)


entrada = {'nuprocessounificadoformatado' : nuprocessounificadoformatado,
           'versao_parser': 5.1}

encerrar = False
if not driver:
    driver = Chrome(os.path.join(settings.STATICFILES_DIRS[0], 'chromedriver'))
    driver.get('http://esaj.tjsp.jus.br/sajcas/login?usuarioIntegracao=46392072000122&senhaIntegracao=sn1305sefd')
    encerrar = True

    
driver.get(f'https://esaj.tjsp.jus.br/cpopg/search.do?cbPesquisa=NUMPROC&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsultaNuUnificado={nuprocessounificadoformatado}')
sp = bs(driver.page_source, 'html5lib')

if "mostrar: 'true'" in str(sp):
    entrada['segredo_de_justica'] = True

else:
    if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:

        entrada['digital'] = True

        crit = re.compile(r'requestScope = (.*?);\s+?var', re.I | re.DOTALL)
        parametro = re.search(r'.codigo=(.*?)&', driver.current_url).group(1)

        

        driver.get(f'https://esaj.tjsp.jus.br/cpopg/abrirPastaDigital.do?processo.codigo={parametro}')

        j = json.loads(re.search(crit, driver.page_source).group(1))

        coleta = []
        c_assinatura = re.compile(
            'p\s{,4}a\s{,4}r\s{,4}a\s{,4}c\s{,4}o\s{,4}n\s{,4}f\s{,4}e\s{,4}r\s{,4}i\s{,4}r\s{,4}.*?fls.{,4}\d+',
            re.I | re.DOTALL)

        for linha in j:
            for sublinha in linha['children']:

                rp = {
                    'indicepagina': sublinha['data']['indicePagina'],
                    'title': linha['data']['title'],
                    'cddocumento': linha['data']['cdDocumento'],
                    'dtinclusao': linha['data']['dtInclusao'],
                    'cdtipodocdigital': linha['data']['cdTipoDocDigital'],
                    'parametros': sublinha['data']['parametros'].replace('amp;', ''),
                }

                if documentos:

                    r_doc = s.get('https://esaj.tjsp.jus.br/pastadigital/getPDF.do?' + rp['parametros'], verify=False)
                    rp['link_doc'] = r_doc.url

                    tempdoc = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False )
                    with open(tempdoc.name, 'wb') as f:
                        f.write(r_doc.content)
                    t = textract.process(tempdoc.name, encoding='latin1').decode('latin1')
                    t = re.sub(crit_assinatura, '', t).strip()

                    if len(t) > 100:
                        t = t.encode(encoding='latin1', errors='replace').decode(encoding='latin1')
                        rp['conteudo'] = t

                    os.remove(tempdoc.name)

                coleta += [rp]

        if len(coleta) > 0:
            entrada['documentos'] = coleta


    # diversos

    for item in sp.findAll(attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
        if item.get('for') == 'tag.dados.rotulo.principal':
            entrada['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

        elif item.get_text() == 'Distribuição:':
            entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            rp = item.findNext('span').findNext()
            if rp.find('span') is not None:
                entrada['vara'] = rp.find('span').get_text().strip()

        elif item.get_text() == 'Apensado ao:':
            entrada[item.get_text().strip().replace(':', '')] = item.findNext(
                attrs={'class': 'processoPaiApenso'}).get_text().strip()

        elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
            entrada['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
        else:
            entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

        if item.get_text() == 'Recebido em:':
            entrada['vara'] = item.findNext('span').findNext('span').get_text()

    for i in sp.findAll('span', attrs={'class': 'labelClass'}):
        entrada[i.get_text().replace(':', '')] = i.next.next.strip()

    # extinto_suspenso
    if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
        entrada['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

    if 'Distribuição' in entrada.keys():

        entrada['data_inicial'] = entrada['Distribuição'][:10]
    elif 'Recebido em' in entrada.keys():
        entrada['data_inicial'] = entrada['Recebido em'][:10]

    if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
        entrada['tramitacao_prioritaria'] = 'Tramitação prioritária'
    if 'Valor da ação' in entrada.keys():
        entrada['Valor da ação'] = re.sub(r'\s\s+', ' ', entrada['Valor da ação'])
        if re.match(r'(.*?)\s', entrada['Valor da ação']) is not None:
            entrada['moeda'] = re.match(r'(.*?)\s', entrada['Valor da ação']).group(1)
        if re.search(r'\s(.*)', entrada['Valor da ação']) is not None:
            rp = re.search(r'\s(.*)', entrada['Valor da ação']).group(1).strip()
            entrada['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

        del entrada['Valor da ação']

    # partes
    coleta = []
    controle = []

    for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top','style': 'padding-bottom: 5px'}):
        rp = {}
        if item.findNext('td').next.strip() not in controle:
            controle += [item.findNext('td').next.strip()]
            rp['nome'] = item.findNext('td').next.strip()
            rp['posicao'] = item.get_text().strip().replace(':', '')

            procuradores = []

            for subitem in item.findNextSibling().findAll('span'):
                classe = subitem.get_text().replace(':', '').strip()
                nome = subitem.next.next.strip()
                procuradores += [[classe, nome]]

            classes = list(set([i[0] for i in procuradores]))
            subcoleta = {}

            for classe in classes:
                nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                subcoleta[classe] = nome_procurador

            if len(subcoleta) > 0:
                rp['procuradores'] = subcoleta
            coleta += [rp]

    if len(coleta) > 0:
        entrada['partes'] = coleta

    # movimentacoes
    coleta = []
    lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
    r = list(range(len(lista) - 1, -1, -1))

    for n, item in enumerate(sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})[::-1]):

        mov = {'ordem': n,
               'data': item.get_text().strip()}

        if len(item.findNext('td').findChildren()) > 0:
            mov['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
            mov['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
        else:

            mov['tag'] = item.findNext('td').findNext('td').next.strip()
            mov['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
        coleta += [mov]

    if len(coleta) > 0:
        entrada['movimentacoes'] = coleta

entrada = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in entrada.items() if v}

# formatacoes e salvar

if 'data_inicial' in entrada.keys():

    entrada['data_inicial'] = datetime.datetime.strptime(entrada['data_inicial'], '%d/%m/%Y').date()

informacao = Informacoes(**{k: v for k, v in entrada.items() if k not in ['movimentacoes', 'partes', 'documentos']})
informacao.save()

if 'documentos' in entrada.keys():
    entrada['documentos'] = [{k: (v if k !=  'dtinclusao' else datetime.datetime.strptime(v, '%d/%m/%Y %H:%M:%S')) for k,v in i.items() } for i in entrada['documentos'] ]

    lista = [Documentos(**i, informacao=informacao) for i in entrada['documentos']]

    try:
        Documentos.objects.bulk_create(lista)
    except:
        for c in lista:
            try:
                c.save()
            except:
                pass

if 'partes' in entrada.keys():
    lista = [Partes(**i, informacao=informacao) for i in entrada['partes']]
    Partes.objects.bulk_create(lista)

if 'movimentacoes' in entrada.keys():
    entrada['movimentacoes'] = [{k: (v if k !=  'data' else datetime.datetime.strptime(v, '%d/%m/%Y').date()) for k,v in i.items() } for i in entrada['movimentacoes'] ]
    lista = [Movimentacoes(**i, informacao=informacao) for i in entrada['movimentacoes']]
    Movimentacoes.objects.bulk_create(lista)




In [7]:
entrada

{'nuprocessounificadoformatado': '1058720-17.2019.8.26.0053',
 'versao_parser': 5.1,
 'digital': True,
 'documentos': [{'indicepagina': 1,
   'title': 'Petição',
   'cddocumento': '135891024',
   'dtinclusao': datetime.datetime(2019, 10, 29, 13, 26, 6),
   'cdtipodocdigital': '9500',
   'parametros': 'nuSeqRecurso=00000&nuProcesso=1058720-17.2019.8.26.0053&cdDocumentoOrigem=0&cdDocumento=135891024&conferenciaDocEdigOriginal=false&nmAlias=PG5JM&origemDocumento=P&nuPagina=1&numInicial=1&tpOrigem=2&cdTipoDocDigital=9500&flOrigem=P&deTipoDocDigital=Peti%E7%E3o&cdProcesso=1H000GAT50000&cdFormatoDoc=9&cdForo=53&idDocumento=135891024-1-1&numFinal=74&sigiloExterno=N'},
  {'indicepagina': 75,
   'title': 'Documento 1',
   'cddocumento': '135891029',
   'dtinclusao': datetime.datetime(2019, 10, 29, 13, 26, 8),
   'cdtipodocdigital': '9899',
   'parametros': 'nuSeqRecurso=00000&nuProcesso=1058720-17.2019.8.26.0053&cdDocumentoOrigem=0&cdDocumento=135891029&conferenciaDocEdigOriginal=false&nmAlias=

In [1]:
from bs4 import BeautifulSoup as bs
import datetime
import re
import json
from unidecode import unidecode
import io
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from api.models import *
from django.conf import settings
import os
import textract
import tempfile
import time
import requests



def get_esaj_logged_session():
    print('AAA')
    s = requests.session()

    url = 'https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check'

    r = s.get(url, verify=False)

    sp = bs(r.content, 'html5lib')

    execution = sp.find('input', attrs={'id': 'flowExecutionKey'}).get('value')

    h = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Content-Length': '4531',
        'Content-Type': 'application/x-www-form-urlencoded',
        'DNT': '1',
        'Host': 'esaj.tjsp.jus.br',
        'Origin': 'https://esaj.tjsp.jus.br',
        'Upgrade-Insecure-Requests': '1',

    }

    params = {'service': 'https://esaj.tjsp.jus.br/esaj/j_spring_cas_security_check'}

    data = {
        'username': '013.787.440-55',
        'password': 'rasengan',
        'execution': execution,
        'lt': '',
        '_eventId': 'submit',
        'pbEntrar': 'Entrar',
        'signature': '',
        'certificadoSelecionado': '',
        'certificado': ''}

    s.post(url, data=data, headers=h, params=params)

    return s

nuprocessounificadoformatado = '1058720-17.2019.8.26.0053'

crit_assinatura = re.compile('Este documento é cópia do original.*? e código .{,8}\.',re.DOTALL)


entrada = {'nuprocessounificadoformatado' : nuprocessounificadoformatado,
           'versao_parser': 5.1}

params = {'cbPesquisa': 'NUMPROC',
          'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
          'dadosConsulta.valorConsultaNuUnificado': nuprocessounificadoformatado}

s = get_esaj_logged_session()

r = s.get('https://esaj.tjsp.jus.br/cpopg/search.do', params=params, verify=False)
print(r.url)
sp = bs(r.content, 'html5lib')

if "mostrar: 'true'" in str(sp):
    entrada['segredo_de_justica'] = True

else:

    if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:

        entrada['digital'] = True

        crit = re.compile(r'requestScope = (.*?);\s+?var', re.I | re.DOTALL)
        parametro = re.search(r'.codigo=(.*?)&', r.url).group(1)

        s.get('https://esaj.tjsp.jus.br/cpopg/show.do?processo.foro=53&processo.codigo=%s&gateway=true' % parametro)

        r = s.get('https://esaj.tjsp.jus.br/cpopg/abrirPastaDigital.do?processo.codigo=%s' % parametro, verify=False)

        j = json.loads(re.search(crit, r.text).group(1))

        coleta = []
        coleta_mandados = []

        c_assinatura = re.compile(
            'p\s{,4}a\s{,4}r\s{,4}a\s{,4}c\s{,4}o\s{,4}n\s{,4}f\s{,4}e\s{,4}r\s{,4}i\s{,4}r\s{,4}.*?fls.{,4}\d+',
            re.I | re.DOTALL)

        for linha in j:
            for sublinha in linha['children']:

                rp = {
                    'indicepagina': sublinha['data']['indicePagina'],
                    'title': linha['data']['title'],
                    'cddocumento': linha['data']['cdDocumento'],
                    'dtinclusao': linha['data']['dtInclusao'],
                    'cdtipodocdigital': linha['data']['cdTipoDocDigital'],
                    'parametros': sublinha['data']['parametros'].replace('amp;', ''),
                }

                if documentos:

                    r_doc = s.get('https://esaj.tjsp.jus.br/pastadigital/getPDF.do?' + rp['parametros'], verify=False)
                    rp['link_doc'] = r_doc.url

                    tempdoc = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False )
                    with open(tempdoc.name, 'wb') as f:
                        f.write(r_doc.content)
                    t = textract.process(tempdoc.name, encoding='latin1').decode('latin1')
                    t = re.sub(crit_assinatura, '', t).strip()

                    if len(t) > 100:
                        t = t.encode(encoding='latin1', errors='replace').decode(encoding='latin1')
                        rp['conteudo'] = t


                        if rp['title'] == 'Mandados':

                            coleta_mandados+=  [{'indicepagina': rp['indicepagina'],
                                                'codigo_validacao' : re.search('\*(\d+)\*', t).group(1),
                                                'url' : r_doc.url}]

                    os.remove(tempdoc.name)

                coleta += [rp]

        if len(coleta) > 0:
            entrada['documentos'] = coleta
        if len(coleta_mandados)>0:
            entrada['mandados'] = coleta_mandados

    # diversos

    for item in sp.findAll(attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
        if item.get('for') == 'tag.dados.rotulo.principal':
            entrada['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

        elif item.get_text() == 'Distribuição:':
            entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            rp = item.findNext('span').findNext()
            if rp.find('span') is not None:
                entrada['vara'] = rp.find('span').get_text().strip()

        elif item.get_text() == 'Apensado ao:':
            entrada[item.get_text().strip().replace(':', '')] = item.findNext(
                attrs={'class': 'processoPaiApenso'}).get_text().strip()

        elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
            entrada['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
        else:
            entrada[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

        if item.get_text() == 'Recebido em:':
            entrada['vara'] = item.findNext('span').findNext('span').get_text()

    for i in sp.findAll('span', attrs={'class': 'labelClass'}):
        entrada[i.get_text().replace(':', '')] = i.next.next.strip()

    # extinto_suspenso
    if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
        entrada['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

    if 'Distribuição' in entrada.keys():

        entrada['data_inicial'] = entrada['Distribuição'][:10]
    elif 'Recebido em' in entrada.keys():
        entrada['data_inicial'] = entrada['Recebido em'][:10]

    if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
        entrada['tramitacao_prioritaria'] = 'Tramitação prioritária'
    if 'Valor da ação' in entrada.keys():
        entrada['Valor da ação'] = re.sub(r'\s\s+', ' ', entrada['Valor da ação'])
        if re.match(r'(.*?)\s', entrada['Valor da ação']) is not None:
            entrada['moeda'] = re.match(r'(.*?)\s', entrada['Valor da ação']).group(1)
        if re.search(r'\s(.*)', entrada['Valor da ação']) is not None:
            rp = re.search(r'\s(.*)', entrada['Valor da ação']).group(1).strip()
            entrada['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

        del entrada['Valor da ação']

    # partes
    coleta = []
    controle = []

    for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top','style': 'padding-bottom: 5px'}):
        rp = {}
        if item.findNext('td').next.strip() not in controle:
            controle += [item.findNext('td').next.strip()]
            rp['nome'] = item.findNext('td').next.strip()
            rp['posicao'] = item.get_text().strip().replace(':', '')

            procuradores = []

            for subitem in item.findNextSibling().findAll('span'):
                classe = subitem.get_text().replace(':', '').strip()
                nome = subitem.next.next.strip()
                procuradores += [[classe, nome]]

            classes = list(set([i[0] for i in procuradores]))
            subcoleta = {}

            for classe in classes:
                nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                subcoleta[classe] = nome_procurador

            if len(subcoleta) > 0:
                rp['procuradores'] = subcoleta
            coleta += [rp]

    if len(coleta) > 0:
        entrada['partes'] = coleta

    # movimentacoes
    coleta = []
    lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
    r = list(range(len(lista) - 1, -1, -1))

    for n, item in enumerate(sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})[::-1]):

        mov = {'ordem': n,
               'data': item.get_text().strip()}

        if len(item.findNext('td').findChildren()) > 0:
            mov['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
            mov['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
        else:

            mov['tag'] = item.findNext('td').findNext('td').next.strip()
            mov['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
        coleta += [mov]

    if len(coleta) > 0:
        entrada['movimentacoes'] = coleta

entrada = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in entrada.items() if v}

# formatacoes e salvar

if 'data_inicial' in entrada.keys():

    entrada['data_inicial'] = datetime.datetime.strptime(entrada['data_inicial'], '%d/%m/%Y').date()

informacao = Informacoes(**{k: v for k, v in entrada.items() if k not in ['movimentacoes', 'partes', 'documentos']})
informacao.save()

if 'documentos' in entrada.keys():
    entrada['documentos'] = [{k: (v if k !=  'dtinclusao' else datetime.datetime.strptime(v, '%d/%m/%Y %H:%M:%S')) for k,v in i.items() } for i in entrada['documentos'] ]

    lista = [Documentos(**i, informacao=informacao) for i in entrada['documentos']]

    try:
        Documentos.objects.bulk_create(lista)
    except:
        for c in lista:
            try:
                c.save()
            except:
                pass

if 'partes' in entrada.keys():
    lista = [Partes(**i, informacao=informacao) for i in entrada['partes']]
    Partes.objects.bulk_create(lista)

if 'movimentacoes' in entrada.keys():
    entrada['movimentacoes'] = [{k: (v if k !=  'data' else datetime.datetime.strptime(v, '%d/%m/%Y').date()) for k,v in i.items() } for i in entrada['movimentacoes'] ]
    lista = [Movimentacoes(**i, informacao=informacao) for i in entrada['movimentacoes']]
    Movimentacoes.objects.bulk_create(lista)

if 'mandados' in entrada.keys():
    lista = [Mandados(**i, informacao=informacao) for i in entrada['mandados']]
    Mandados.objects.bulk_create(lista)




AAA
https://esaj.tjsp.jus.br/cpopg/search.do?cbPesquisa=NUMPROC&dadosConsulta.tipoNuProcesso=UNIFICADO&dadosConsulta.valorConsultaNuUnificado=1058720-17.2019.8.26.0053


In [21]:
s = requests.session()

url = 'https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check'

r = s.get(url, verify=False)

sp = bs(r.content, 'html5lib')

execution = sp.find('input', attrs={'id': 'flowExecutionKey'}).get('value')

h = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Length': '4531',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Host': 'esaj.tjsp.jus.br',
    'Origin': 'https://esaj.tjsp.jus.br',
    'Upgrade-Insecure-Requests': '1',

}

params = {'service': 'https://esaj.tjsp.jus.br/esaj/j_spring_cas_security_check'}

data = {
    'username': '013.787.440-55',
    'password': 'rasengan',
    'execution': execution,
    'lt': '',
    '_eventId': 'submit',
    'pbEntrar': 'Entrar',
    'signature': '',
    'certificadoSelecionado': '',
    'certificado': ''}

# s.post(url, data=data, headers=h, params=params)

# s

In [6]:
import requests
from bs4 import BeautifulSoup as bs
# def get_esaj_logged_session():

s = requests.session()

# url = 'https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check'

# r = s.get(url, verify=False)

# sp = bs(r.content, 'html5lib')

# execution = sp.find('input', attrs={'id': 'flowExecutionKey'}).get('value')

# h = {
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
#     'Cache-Control': 'max-age=0',
#     'Connection': 'keep-alive',
#     'Content-Length': '4531',
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'DNT': '1',
#     'Host': 'esaj.tjsp.jus.br',
#     'Origin': 'https://esaj.tjsp.jus.br',
#     'Upgrade-Insecure-Requests': '1',

# }

# params = {'service': 'https://esaj.tjsp.jus.br/esaj/j_spring_cas_security_check'}

# data = {
#     'username': '013.787.440-55',
#     'password': 'rasengan',
#     'execution': execution,
#     'lt': '',
#     '_eventId': 'submit',
#     'pbEntrar': 'Entrar',
#     'signature': '',
#     'certificadoSelecionado': '',
#     'certificado': ''}

# s.post(url, data=data, headers=h, params=params)

r = s.get('http://esaj.tjsp.jus.br/sajcas/login?usuarioIntegracao=46392072000122&senhaIntegracao=sn1305sefd')

# r = s.get('https://esaj.tjsp.jus.br/cpopg/show.do?processo.foro=53&processo.codigo=1H000GAT50000')

r

<Response [200]>

In [7]:

sp = bs(r.content, 'html5lib')
sp.get_text()

'\n\n\t\n\t\tif(window.sajcas && window.sajcas.usuarioLogadoNoCasServer) {\n\t\t\tvar urlRetornoSistema = \'\';\n\t\t\tif(urlRetornoSistema === \'\') {\n\t\t\t\turlRetornoSistema = window.location.href;\n\t\t\t}\n\n\t\t\tif(urlRetornoSistema.lastIndexOf(\'?\') != -1) {\n\t\t\t\turlRetornoSistema += \'&\';\n\t\t\t} else {\n\t\t\t\turlRetornoSistema += \'?\';\n\t\t\t}\n\t\t\turlRetornoSistema+= \'gateway=true\';\n\t\t\twindow.location.href = urlRetornoSistema;\n\t\t}\n\t\n\n\n\n\n\n\n\n\t\n\t\n\n\n\n\n\n\n\n\n\n\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t\n\n\n\t\n\n\ne-SAJ\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n<!--\n/* botao default means o mais claro, que seria o "bot�o principal" */\n.spwBotaoDefault, .spwBotaoDefault-o { \n\tbackground-image: url(https://esaj.tjsp.jus.br/esaj/tema/padrao/clientes/SP/imagens/layout/fundoBotaoDefault.gif);\n}\n/* o botao secundario, menos provavel de ser clicado, mais escuro */\n.spwBotao, .spwBotao-o { \n\tbackground-image: ur

In [16]:
data

{'username': '013.787.440-55',
 'password': 'rasengan',
 'execution': '3af9abf4-1ea3-4141-b03c-8f2d34dc9d8b_ZXlKaGJHY2lPaUpJVXpVeE1pSjkuU0ZoTVEwRXlaSFZxUkdsTlRXaE1hbkJtU0VzNE0yTXJNM1ZGVkZNeU1YVlhXVlJxYWpKME1DOHhUVWQxV0d4RFMwTlNlRE5yYUZaTlkxSlpiSEo1YnpkWGNGcHJXVU5IYkVoVlZVWmpTWFJsY1dGRU0zUjRWbGhJTUVGVFVUSkhPVmhZVEhFclF6SlNTM2RaY0hWRGFHSmhhRTEwZGtWdVMwNXpkVGwzTkVFeVdHaG9PRXRKTlZwM04zSm1iREZLY2paV01HUllXVmhDUlZCNGJrbERkM05vVEZWUFVrOUROak5zZFZoaVZDdG5aMkZRTDB0S1ExUkhlbk0zVkc1d05XVm1RVE52WlVOTlFXVnlValZXTWk5RFdWSmxSRzU0TVdjdlkyeE9NR1JNU1hSdWJ5dGpSSFZ3UWsxRmRGaEdWV3RGZUdzdllreHZZVVpGVjJKRVNDdE5LMU01UWtac1dDOVViRXM1YXpRd2VtUnlZUzl5T0hwT2RFWnVaVm96TWxKeE5VSkJORkZGUmt4Q1preFRORGN6YW1oMWNGaEtZVEl2UVd3MlZVNVBXWGRoYkZWMVRucERWWEJpVTNodE5qTkJPRFF6WnpJNE1GWnFjRmR3WlhKbFkzWjVaMGRMVTBoaE4ySkZTRVpvVUVsSllrOXpZa2xpTHpRNFQwdEtjVXRtWVdaUU4za3lOemQyY1dnNVYwVTFNV3BoZWpjNWFXRk5Uek16ZEdWa1ZsTm1UalJrZWt4VWJ6SkNUMnhOWXpjMlFXcDJja2RNYlVGMlRrNVJNamhRYWxBdmRuTnBPRFozTVZkbFRtWmFjMVozYlVSWlQzUlJhVmx5ZVhweWN6RlFPVzh3